# Spotify API


In [1]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from spotipy import oauth2
import random
from functools import reduce
import requests
from spotify.spotify_creds import SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, SPOTIFY_USER, SPOTIFY_REDIRECT_URI
import streamlit as st
import streamlit.components.v1 as components

## Authentification


In [2]:
# ID and password
cid = SPOTIFY_CLIENT_ID
secret = SPOTIFY_CLIENT_SECRET
username = SPOTIFY_USER
uri = SPOTIFY_REDIRECT_URI
scope = 'user-read-private user-read-email'

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
# sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="YOUR_APP_CLIENT_ID",
#                                                            client_secret="YOUR_APP_CLIENT_SECRET"))

# results = sp.search(q='bicep', limit=20)
# for idx, track in enumerate(results['tracks']['items']):
#     print(idx, track['name'])

In [5]:
# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cid,
#                                                client_secret=secret,
#                                                redirect_uri=uri,
#                                                scope=scope))

# results = sp.current_user_saved_tracks()
# for idx, item in enumerate(results['items']):
#     track = item['track']
#     print(idx, track['artists'][0]['name'], " – ", track['name'])

In [7]:
def spotify_authentification():
    """
    Api authentification using requests
    """
    AUTH_URL = 'https://accounts.spotify.com/api/token'

    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': cid,
        'client_secret': secret,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token = auth_response_data['access_token']

    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
    }
    return headers

headers = spotify_authentification()

## Database

In [94]:
def artist_track_features(artist):
    """
    This function will provide us with a dataframe with all sounds of an artist 
    and their underlying features.
    
    Inputs :
    artist = <Name of the artist>
    
    Outputs :
    > Dataframe containing the sounds of an artist
    """
    # Fetch artist tracks
    d = {}
    track_results = sp.search(q=artist,limit=50)
    print(track_results)
    for i, t in enumerate(track_results['tracks']['items']):
        track_id = t['id']
        if track_id not in d:
            d[track_id] = {'artist_feature' : t['artists'][0]['name'],
                           'query' : artist,
                           'track_name' : t['name'],
                           'popularity': t['popularity']}    
            d[track_id].update(sp.audio_features(d.keys())[0])
            
            # add genres
            headers = spotify_authentification()
            BASE_URL = 'https://api.spotify.com/v1/'
            re = requests.get(BASE_URL + 'playlists/' + url + '/tracks', headers=headers).json()
            if re != None :
                layer = requests.get(re['artists'][0]['href'], headers=headers).json()
                if 'genres' in layer.keys():
                    d[track_id].update({'genres':layer['genres']})
                else :
                    d[track_id].update({'genres':'None'})
    return pd.DataFrame.from_dict(d, orient='index')


In [95]:
artist_track_features('bicep')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=bicep&type=track&offset=0&limit=50', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/64NhyHqRKYhV0IZylrElWu'}, 'href': 'https://api.spotify.com/v1/artists/64NhyHqRKYhV0IZylrElWu', 'id': '64NhyHqRKYhV0IZylrElWu', 'name': 'TR/ST', 'type': 'artist', 'uri': 'spotify:artist:64NhyHqRKYhV0IZylrElWu'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU

KeyError: 'artists'

In [93]:
headers = spotify_authentification()
BASE_URL = 'https://api.spotify.com/v1/'
url = '37i9dQZF1DX7rOY2tZUw1k'
re = requests.get(BASE_URL + 'playlists/' + url + '/tracks', headers=headers).json()
d = []
re['artists']

KeyError: 'artists'

In [41]:
def playlist_features(playlist):
    """
    This function will provide us with a dataframe with all sounds of an artist 
    and their underlying features.
    
    Inputs :
    artist = <Name of the artist>
    
    Outputs :
    > Dataframe containing the sounds of an artist
    """
    # Fetch artist tracks
    d = {}
    track_results = sp.search(q=playlist, type='track',limit=50)
    for i, t in enumerate(track_results['tracks']['items']):
        track_id = t['id']
        if track_id not in d:
            d[track_id] = {'artist_feature' : t['artists'][0]['name'],
                           'query' : artist,
                           'track_name' : t['name'],
                           'popularity': t['popularity']}    
            d[track_id].update(sp.audio_features(d.keys())[0])

            # add genres
            headers = spotify_authentification()
            BASE_URL = 'https://api.spotify.com/v1/'
            re = requests.get(playlist, headers=headers).json()
            if re != None :
                layer = requests.get(re['artists'][0]['href'], headers=headers).json()
                if 'genres' in layer.keys():
                    d[track_id].update({'genres':layer['genres']})
                else :
                    d[track_id].update({'genres':'None'})
    return pd.DataFrame.from_dict(d, orient='index')

playlist_features('https://open.spotify.com/playlist/556ICk4gRzDknRfWGeQ3x1?si=317b09e2dc4b45f3')

""


## Playlist function

In [9]:
def select_and_play(feeling):
    happiness = sp.playlist(playlist_id = '2oFSJfwxmLUqZUU4SgD0I7?si=671d061cbcfc4810')
    sadness = sp.playlist(playlist_id = '0LbtLlb9G4M7TFi8OsDYMg?si=f6cf8536ee3c4e6f')
    #neutral
    worry = sp.playlist(playlist_id = '06Gh1LqdIfG1HMHKa44B7k?si=ad744fcfa57c4be3') #Same playlist as fear
    love = sp.playlist(playlist_id = '37i9dQZF1DX7rOY2tZUw1k?si=88404786a9ae4cc6')
    anger = sp.playlist(playlist_id = '5c5NfSIO6bMrUSoCZKMudz?si=211e93362660427b')
    #surprise
    fear = sp.playlist(playlist_id = '06Gh1LqdIfG1HMHKa44B7k?si=ad744fcfa57c4be3')#same playlist as worry
    fun = sp.playlist(playlist_id = '7xOHp3ZlSBJNJOgsQwF85S?si=f04c656c9241412b')
    relief = sp.playlist(playlist_id = '37i9dQZF1DWXe9gFZP0gtP?si=cefe9eafc2e14d8d')
    hate = sp.playlist(playlist_id = '4OGYlQGUpMY6EZsmFdiJ1e?si=0252727c471f41e8')
    enthusiasm = sp.playlist(playlist_id = '5t0hWEUHZ89a8jzBulvuvY?si=3dd98c1bb2c24bb2')
    boredom = sp.playlist(playlist_id = '37i9dQZF1DWWQRwui0ExPn?si=575ab13eac8c418f')
    feeling_playlists = [happiness, sadness, worry, love, anger, fear, fun, relief, hate, enthusiasm, boredom]
    d= {'happy':happiness, 'sad':sadness, 'worry':worry, 'love':love, 
        'anger':anger, 'fear':fear, 'fun':fun, 
        'relief': relief, 'hate':hate, 'enthusiasm':enthusiasm,
        'boredom':boredom
       }
    return list(d[feeling].get('external_urls').values())
#     for i in feeling_playlists:
#         if i == str(feeling):
#             return i.get('external_urls').values()
select_and_play('fun')

['https://open.spotify.com/playlist/7xOHp3ZlSBJNJOgsQwF85S']

In [ ]:
def select_and_play_second(feeling):
    happy = '2oFSJfwxmLUqZUU4SgD0I7?si=671d061cbcfc4810'
    sad = '0LbtLlb9G4M7TFi8OsDYMg?si=f6cf8536ee3c4e6f'
    #neutral
    worry = '06Gh1LqdIfG1HMHKa44B7k?si=ad744fcfa57c4be3' #Same playlist as fear
    love = '37i9dQZF1DX7rOY2tZUw1k?si=88404786a9ae4cc6'
    anger = '5c5NfSIO6bMrUSoCZKMudz?si=211e93362660427b'
    #surprise
    fear = '06Gh1LqdIfG1HMHKa44B7k?si=ad744fcfa57c4be3' #same playlist as worry
    fun = '7xOHp3ZlSBJNJOgsQwF85S?si=f04c656c9241412b'
    relief = '37i9dQZF1DWXe9gFZP0gtP?si=cefe9eafc2e14d8d'
    hate = '4OGYlQGUpMY6EZsmFdiJ1e?si=0252727c471f41e8'
    enthusiasm = '5t0hWEUHZ89a8jzBulvuvY?si=3dd98c1bb2c24bb2'
    boredom = '37i9dQZF1DWWQRwui0ExPn?si=575ab13eac8c418f'
    d= {'happy':happy, 'sad':sad, 'worry':worry, 'love':love, 
        'anger':anger, 'fear':fear, 'fun':fun, 
        'relief': relief, 'hate':hate, 'enthusiasm':enthusiasm,
        'boredom':boredom
       }
    return d[feeling]
#     for i in feeling_playlists_2:
#         if i == feeling:
#             return i

In [ ]:
select_and_play_second('sad')

In [ ]:
def select_and_play(feeling):
    happy = 'https://open.spotify.com/playlist/556ICk4gRzDknRfWGeQ3x1?si=317b09e2dc4b45f3'
    sad = 'https://open.spotify.com/playlist/0dRxDrR1PfZMlVbfnuBRbR?si=1c1095aff5cf4b24'
    anger = 'https://open.spotify.com/playlist/7FjP7MbRgFYFdv5avuhiBI?si=4289f7bbaa1243af'
    enthusiasm = 'https://open.spotify.com/playlist/4JsAKbWk4AoBcfpSs2afOM?si=d578284db05845ca'
    empty = 'https://open.spotify.com/playlist/77OpFLSdLy3nC9huQIXlxk?si=205fa8c2b1d24e8d'
    boredom = 'https://open.spotify.com/playlist/2rA0wLILuvNuLhAacn4kth?si=9ec06d60f5b14c53'
    worry = 'https://open.spotify.com/playlist/5Dt93qIXccZvZbU6r3oIbs?si=061a1e6b521b422f'
    love = 'https://open.spotify.com/playlist/73KuPUAtOecLDAetRn80TW?si=dda2764abcde4498'
    surprise = 'https://open.spotify.com/playlist/0BaRZECQEqp4zDd0Njzlj1?si=d943c1a27fec45df'
    fun = 'https://open.spotify.com/playlist/7HwdXmzNKXBzTAisOKYVsJ?si=6f73860860db49d1'
    relief = 'https://open.spotify.com/playlist/2zaAFRdI6lEaX8Esc11XPZ?si=b4ea70e95eca4506'
    hate = 'https://open.spotify.com/playlist/6vdOF3ZRqiYimqEm4lk98V?si=eb11df3e48b44e4e'
    neutral = 'https://open.spotify.com/playlist/5pSdjjPHbXpbqFJGf31Ksn?si=8a2e9c4bf0e04ef5'

    d= {'happy':happy, 'sad':sad, 'worry':worry, 'love':love, 
        'anger':anger, 'fun':fun, 'relief': relief, 'empty':empty,
        'hate':hate, 'enthusiasm':enthusiasm, 'surprise':surprise,
        'boredom':boredom, 'neutral':neutral
       }
    
    mood = d[feeling]
    
    return components.html(
                f'<iframe src="{mood}" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>',
                height=600
                )


In [ ]:
components.html(
        f'<iframe src="https://open.spotify.com/playlist/2rA0wLILuvNuLhAacn4kth?si=9ec06d60f5b14c53" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>',
    height=600
)

## Putting it together

In [ ]:
try:
    token = util.prompt_for_user_token(username, scope) # add scope
except (AttributeError, JSONDecodeError):
    os.remove(f".cache-{username}")
    token = util.prompt_for_user_token(username, scope) # add scope

# Create our spotify object with permissions
spotifyObject = spotipy.Spotify(auth=token)

In [ ]:
# Current track information
track = spotifyObject.current_user_playing_track()
artist = track['item']['artists'][0]['name']
track = track['item']['name']

if artist != "":
    print("Currently playing " + artist + " - " + track)

In [ ]:
#User information
user = spotifyObject.current_user()
displayName = user['display_name']
followers = user['followers']['total']

In [ ]:
# Loop
while True:
    # Main Menu
    print()
    print(">>> Welcome to Spotipy " + displayName + "!")
    print(">>> You have " + str(followers) + " followers.")
    print()
    print("0 - Search for an artist")
    print("1 - exit")
    print()
    choice = input("Your choice: ")

    if choice == "0":
        print()
        searchQuery = input("Ok, what's their name?: ")
        print()

        # Get search results
        searchResults = spotifyObject.search(searchQuery,1,0,"artist")

        # Artist details
        artist = searchResults['artists']['items'][0]
        print(artist['name'])
        print(str(artist['followers']['total']) + " followers")
        print(artist['genres'][0])
        print()
        #webbrowser.open(artist['images'][0]['url'])
        artistID = artist['id']


        # Album and track details
        trackURIs = []
        trackArt = []
        z = 0

        # Extract album data
        albumResults = spotifyObject.artist_albums(artistID)
        albumResults = albumResults['items']

        for item in albumResults:
            print("ALBUM: " + item['name'])
            albumID = item['id']
            albumArt = item['images'][0]['url']

            # Extract track data
            trackResults = spotifyObject.album_tracks(albumID)
            trackResults = trackResults['items']

            for item in trackResults:
                print(str(z) + ": " + item['name'])
                trackURIs.append(item['uri'])
                trackArt.append(albumArt)
                z+=1
            print()

        # See album art
        while True:
            songSelection = input("Enter a song number to see album art and play the song (x to exit): ") # and play the song
            if songSelection == "x":
                break
            trackSelectionList = []
            trackSelectionList.append(trackURIs[int(songSelection)])
            spotifyObject.start_playback(deviceID, None, trackSelectionList) # added
            #webbrowser.open(trackArt[int(songSelection)])

    if choice == "1":
        break
